In [1]:
#from langchain_experimental.agents import create_pandas_dataframe_agent
import os
import json
import openai
from openai import AzureOpenAI
#from langchain.llms import AzureOpenAI
#from langchain_openai import AzureChatOpenAI
import pandas as pd 
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
from causalnex.structure import StructureModel
    
sm_manual = StructureModel()
sm_manual.add_edges_from(
    [
    ('age', 'chol'),
    ('age', 'trestbps'),
    ('sex', 'chol'),
    ('sex', 'trestbps'),
    ('sex', 'target'),
    ('cp', 'target'),
    ('trestbps', 'target'),
    ('chol', 'target'),
    ('fbs', 'target'),
    ('restecg', 'target'),
    ('thalach', 'target'),
    ('thalach', 'exang'),
    ('exang', 'target'),
    ('oldpeak', 'target'),
    ('oldpeak', 'exang'),
    ('slope', 'target'),
    ('slope', 'exang')
    ],
    origin="expert",
)

In [3]:
df = pd.read_csv("heart_statlog_cleveland_hungary_final.csv")

In [4]:
df =  df.dropna()
df = df[df['chol'] !=0]
df = df[df['trestbps'] !=0]

In [5]:
from causalnex.network import BayesianNetwork
from causalnex.inference import InferenceEngine

bn = BayesianNetwork(sm_manual)

# Updating the model on the whole dataset
bn.fit_cpds(df, method="BayesianEstimator", bayes_prior="K2")

KeyError: 'age'

In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import CausalInference

In [ ]:
# Discretize continuous variables
df['age'] = pd.cut(df['age'], bins=5, labels=False)
df['chol'] = pd.cut(df['chol'], bins=5, labels=False)
df['trestbps'] = pd.cut(df['trestbps'], bins=5, labels=False)
df['thalach'] = pd.cut(df['thalach'], bins=5, labels=False)
df['oldpeak'] = pd.cut(df['oldpeak'], bins=5, labels=False)

In [ ]:
# Discretize continuous variables and store the bin edges
age_bins = pd.cut(df['age'], bins=5, labels=False, retbins=True)
chol_bins = pd.cut(df['chol'], bins=5, labels=False, retbins=True)
trestbps_bins = pd.cut(df['trestbps'], bins=5, labels=False, retbins=True)
thalach_bins = pd.cut(df['thalach'], bins=5, labels=False, retbins=True)
oldpeak_bins = pd.cut(df['oldpeak'], bins=5, labels=False, retbins=True)

In [ ]:
df['chol'].value_counts()

In [ ]:
model = BayesianNetwork([("age", "chol"),
    ("age", "trestbps"),
    ("sex", "chol"),
    ("sex", "trestbps"),
    ("sex", "target"),
    ("cp", "target"),
    ("trestbps", "target"),
    ("chol", "target"),
    ("fbs", "target"),
    ("restecg", "target"),
    ("thalach", "target"),
    ("thalach", "exang"),
    ("exang", "target"),
    ("oldpeak", "target"),
    ("oldpeak", "exang"),
    ("slope", "target"),
    ("slope", "exang")
 ])


In [ ]:
print(df.shape)
print(df.head())
print(df.isnull().sum())

In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# Filter the DataFrame to find a row where target is 1
target_1_row = df[df['target'] == 1].iloc[1]

# Create an evidence dictionary from the selected row
evidence = {
    'age': target_1_row['age'],
    'sex': target_1_row['sex'],
    'chol': target_1_row['chol'],
    'trestbps': target_1_row['trestbps'],
    'cp': target_1_row['cp'],
    'fbs': target_1_row['fbs'],
    'restecg': target_1_row['restecg'],
    'thalach': target_1_row['thalach'],
    'exang': target_1_row['exang'],
    'oldpeak': target_1_row['oldpeak'],
    'slope': target_1_row['slope']
}

print(evidence)

In [ ]:
data_sample = df.sample(n=100, random_state=42)

In [ ]:
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import BeliefPropagation


# Fit the model to the data
model.fit(data_sample, estimator=MaximumLikelihoodEstimator)

# Initialize the BeliefPropagation object
belief_propagation = BeliefPropagation(model)

# Set the evidence for the observed variables
#evidence = {'age': 45, 'sex': 1, 'chol': 200}

# Perform approximate inference
# Example: What would be the effect on 'target' if 'chol' was set to 150?
belief_propagation.do_intervention({'chol': 190})
approximate_result = belief_propagation.query(variables=['target'], evidence=evidence)

# Print the approximate result
print(approximate_result)

In [ ]:
'''import networkx as nx
import matplotlib.pyplot as plt
nx.draw(model)
plt.show()'''

bnlearn

In [6]:
import bnlearn as bn

ModuleNotFoundError: No module named 'bnlearn'

In [ ]:


# Define the structure of the Bayesian Network
edges = [
    ("age", "chol"),
    ("age", "trestbps"),
    ("sex", "chol"),
    ("sex", "trestbps"),
    ("sex", "target"),
    ("cp", "target"),
    ("trestbps", "target"),
    ("chol", "target"),
    ("fbs", "target"),
    ("restecg", "target"),
    ("thalach", "target"),
    ("thalach", "exang"),
    ("exang", "target"),
    ("oldpeak", "target"),
    ("oldpeak", "exang"),
    ("slope", "target"),
    ("slope", "exang")
]

# Create the Bayesian Network model
model = bn.make_DAG(edges)

# Fit the model to the data
model = bn.parameter_learning.fit(model, df)

PyAgrum

In [ ]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum.causal as csl
import pyAgrum.causal.notebook as cslnb
import os
import math
import numpy as np
import scipy.stats

In [ ]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator

# Load the dataset
#data = pd.read_csv('path_to_your_dataset.csv')

# Preprocess the dataset (if necessary)
# For example, handle missing values or discretize continuous variables
# data = data.fillna(method='ffill')  # Example: forward fill missing values

# Define the structure of the Bayesian Network
model = BayesianNetwork([("age", "chol"),
                         ("age", "trestbps"),
                         ("sex", "chol"),
                         ("sex", "trestbps"),
                         ("sex", "target"),
                         ("cp", "target"),
                         ("trestbps", "target"),
                         ("chol", "target"),
                         ("fbs", "target"),
                         ("restecg", "target"),
                         ("thalach", "target"),
                         ("thalach", "exang"),
                         ("exang", "target"),
                         ("oldpeak", "target"),
                         ("oldpeak", "exang"),
                         ("slope", "target"),
                         ("slope", "exang")])

# Fit the model to the data
model.fit(data, estimator=MaximumLikelihoodEstimator)

# Print the learned CPDs
for cpd in model.get_cpds():
    print(cpd)

In [ ]:
# Update 'sex' column
df['sex'] = df['sex'].replace({1: 'Male', 0: 'Female'})
# Update 'cp' column
df['cp'] = df['cp'].replace({1: 'typical angina', 2: 'atypical angina', 3: 'non-anginal pain', 4: 'asymptomatic'})
# Update 'fbs' column
df['fbs'] = df['fbs'].replace({1: 'True', 0: 'False'})
# Update 'restecg' column
df['restecg'] = df['restecg'].replace({0: 'normal', 1: 'ST-T wave abnormality', 2: 'probable or definite left ventricular hypertrophy'})
# Update 'exang' column
df['exang'] = df['exang'].replace({1: 'yes', 0: 'no'})
# Update 'slope' column
df['slope'] = df['slope'].replace({1: 'upsloping', 2: 'flat', 3: 'downsloping'})
# Update 'target' column
df['target'] = df['target'].replace({1: 'yes', 0: 'no'})

In [ ]:
df_risk = df[df['target'] == 'yes'].reset_index(drop=True)

In [ ]:

def read_data():
    dict_strings = []
    df = pd.read_csv("heart_statlog_cleveland_hungary_final.csv")
    # Update 'sex' column
    df['sex'] = df['sex'].replace({1: 'Male', 0: 'Female'})
    # Update 'cp' column
    df['cp'] = df['cp'].replace({1: 'typical angina', 2: 'atypical angina', 3: 'non-anginal pain', 4: 'asymptomatic'})
    # Update 'fbs' column
    df['fbs'] = df['fbs'].replace({1: 'True', 0: 'False'})
    # Update 'restecg' column
    df['restecg'] = df['restecg'].replace({0: 'normal', 1: 'ST-T wave abnormality', 2: 'probable or definite left ventricular hypertrophy'})
    # Update 'exang' column
    df['exang'] = df['exang'].replace({1: 'yes', 0: 'no'})
    # Update 'slope' column
    df['slope'] = df['slope'].replace({1: 'upsloping', 2: 'flat', 3: 'downsloping'})
    # Update 'target' column
    df['target'] = df['target'].replace({1: 'yes', 0: 'no'})
    df_risk = df[df['target'] == 'yes'].reset_index(drop=True)
    for i in range(len(df_risk)):
        patient_records = df_risk.iloc[i]
        dict_string = json.dumps(patient_records.to_dict())
        dict_strings.append(dict_string)
    return dict_strings


In [ ]:
Dataset_Description = """age: This is the patient’s age, measured in years.
10
2. sex: This variable indicates the patient’s gender, with 1 signifying
Male and 0 signifying Female.
3. cp: This attribute describes the kind of chest discomfort experienced
by the patient, with 1 indicating typical angina, 2 indicating atypical
angina, 3 indicating non-anginal pain, and four indicating asymptomatic.
4. trestbps: Upon hospital admission, the patient’s resting blood pressure
(in mmHg).
5. chol: This variable measures the patient’s cholesterol level in mg/dl.
6. fbs: This attribute indicates whether the patient’s fasting blood sugar
level exceeds 120 mg/dl (1 = True; 0 = False).
7. restecg: This variable represents the resting electrocardiographic measurement,
with 0 indicating normal, 1 indicating ST-T wave abnormality,
and 2 indicating probable or definite left ventricular hypertrophy
by Estes’ criteria.
8. thalach: This attribute denotes the maximum heart rate the individual
achieves during exercise.
9. exang: This variable indicates whether the patient experienced angina
caused by exercise (1 = yes; 0 = no).
10. oldpeak: This attribute refers to the ST depression induced by exercise
relative to rest (‘ST’ refers to positions on the ECG plot).
11. slope: This variable describes the slope of the peak exercise ST segment
(1: upsloping, 2: flat, 3: downsloping).
12. target: This attribute indicates whether the patient has heart disease/
CVD (0 = no, 1 = yes)."""

In [ ]:
import openai
import json
import os
import concurrent.futures
import time

openai.api_type = "azure"
openai.api_base = "https://dap-open-ai-swc.openai.azure.com/"
openai.api_version = "2023-09-15-preview"
openai.api_key = "1fc75eeca3934ae29b46d59cfda85968"

client = AzureOpenAI( azure_endpoint = "https://dap-open-ai-swc.openai.azure.com/", 
                    api_key= "1fc75eeca3934ae29b46d59cfda85968",  
                    api_version="2023-05-15"
                    )

    
def create_patient_description (patient_record):
    # Convert the patient record to a dictionary string
    
    #print(i)
    dict_string = json.dumps(patient_record)

    # Create the message text
    message_text = [{
        "role": "system",
        "content": f"""You are a helpful assistant. Here is the dataset description {Dataset_Description}.
        Describe this patient info
        ---
        {dict_string} 
        ---
        in plain english for me. Be precise and concise.
        """}]

    # Create the chat completion
    completion = client.chat.completions.create(
        model="gpt-4-32k-deployment",
        messages=message_text,
        temperature=0.0,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    # Return the generated description
    return completion.choices[0].message.content




def create_patient_description_with_retry(patient_record):
    retries = 0
    while True:
        try:
            return create_patient_description(patient_record)
        except Exception as e:
            if '429' in str(e):
                print(f"Rate limit exceeded. Retrying after {2**retries} seconds.")
                time.sleep(2**retries)
                retries += 1
            else:
                print(f"An error occurred: {e}")
                return None

def create_patient_descriptions(patient_records):
    # Create a thread pool executor
    with concurrent.futures.ThreadPoolExecutor() as executor:
        patient_descriptions = list(executor.map(create_patient_description_with_retry, patient_records))

    return patient_descriptions

In [ ]:
patient_records = read_data()
patient_descriptions = create_patient_descriptions(patient_records)

In [ ]:
len(patient_descriptions)

In [ ]:
patient_info = patient_descriptions[11]

In [ ]:
patient_info

In [ ]:
 def create_patient_cf (patient_record):
    # Convert the patient record to a dictionary string 
    client = AzureOpenAI( azure_endpoint = "https://dap-open-ai-swc.openai.azure.com/", 
                        api_key= "1fc75eeca3934ae29b46d59cfda85968",  
                        api_version="2023-05-15"
                        )

    message_text = [{"role":"system",
        "content":f"""You are a helpful assistant in Counterfactual Reasoning.
        Here are the patient details:
        --- 
        {patient_record}
        ---
        What should the patient do specifically to prevent heart disease?
        Answer in precise comma separated manner in the following format:
        ---
        {{"Preventive Measures": ["(measure1:value1), (measure2:value2),..."]}}
        ---
        Restrict the preventive measures based on the patient's health data provided to you. Restrict actionable insights to Cholestrol, Resting Blood Pressure, Max Heart Rate and fasting blood sugar levels.
        Here are the constraints. The counterfactual cholestrol should be lesser than patient's Cholestrol cannot be greater than 200.
        Counterfactual Resting Blood Pressure and should be lesser than patient's Resting Blood Pressure and cannot be greater than 120. 
        Counterfactual Max heat rate should be greater than patient's current max heart rate should be lower than 220-Age.
        Give a specific number for each preventive measure. Do not hallucinate. Don't provide general advice and say don't know if you can't provide a specific number.
        """}]

    completion = client.chat.completions.create(
            model="gpt-4-32k-deployment",
            messages=message_text,
            temperature=0.0,
            max_tokens=800,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None
        )
    
    return completion.choices[0].message.content



In [ ]:
 def create_patient_cf_wo_constraints (patient_record):
    # Convert the patient record to a dictionary string 
    client = AzureOpenAI( azure_endpoint = "https://dap-open-ai-swc.openai.azure.com/", 
                        api_key= "1fc75eeca3934ae29b46d59cfda85968",  
                        api_version="2023-05-15"
                        )

    message_text = [{"role":"system",
        "content":f"""You are a helpful assistant in Counterfactual Reasoning.
        Here are the patient details:
        --- 
        {patient_record}
        ---
        What should the patient do specifically to prevent heart disease?
        Answer in precise comma separated manner in the following format:
        ---
        {{"Preventive Measures": ["(measure1:value1), (measure2:value2),..."]}}
        ---
        Restrict the preventive measures based on the patient's health data provided to you. Restrict actionable insights to Cholestrol, Resting Blood Pressure, Max Heart Rate and fasting blood sugar levels.
        Give a specific number for each preventive measure. Do not hallucinate. Don't provide general advice and say don't know if you can't provide a specific number.
        """}]

    completion = client.chat.completions.create(
            model="gpt-4-32k-deployment",
            messages=message_text,
            temperature=0.0,
            max_tokens=800,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None
        )
    
    return completion.choices[0].message.content



In [ ]:
patient_cvd_counterfactual_wo_constraints = []
for idx,patient_record in enumerate(patient_descriptions):
    print(idx)
    patient_cvd_counterfactual_wo_constraints.append(create_patient_cf_wo_constraints(patient_record))

In [ ]:
patient_cvd_counterfactual_wo_constraints

In [ ]:
patient_cvd_counterfactual = []
for idx,patient_record in enumerate(patient_descriptions):
    print(idx)
    patient_cvd_counterfactual.append(create_patient_cf(patient_record))

In [ ]:
# Add 'patient_descriptions' and 'patient_cvd_counterfactual' to 'df_risk'
df_risk['patient_descriptions'] = patient_descriptions
df_risk['patient_cvd_counterfactual'] = patient_cvd_counterfactual

# Save 'df_risk' as a CSV file
df_risk.to_csv('df_risk_3.csv', index=False)

In [ ]:
# Extract the values for each preventive measure and put them into separate columns
df_risk['Cholesterol'] = df_risk['patient_cvd_counterfactual'].str.extract(r'Cholesterol\s*:\s*(\d+)')
df_risk['Resting Blood Pressure'] = df_risk['patient_cvd_counterfactual'].str.extract(r'Resting Blood Pressure\s*:\s*(\d+)')
df_risk['Max Heart Rate'] = df_risk['patient_cvd_counterfactual'].str.extract(r'Max Heart Rate\s*:\s*(\d+)')
df_risk['Fasting Blood Sugar'] = df_risk['patient_cvd_counterfactual'].str.extract(r'Fasting Blood Sugar\s*:\s*(\d+)')

In [ ]:
df_risk.head()

In [ ]:
# Get descriptive statistics of the 'Cholesterol', 'Resting Blood Pressure', and 'Max Heart Rate' columns
df_risk[['Cholesterol', 'Resting Blood Pressure', 'Max Heart Rate']].describe()

In [ ]:
# Convert the list into a DataFrame
df = pd.DataFrame(patient_cvd_counterfactual, columns=['preventive_measures'])

# Split the 'preventive_measures' column into separate columns
df = df['preventive_measures'].str.split(',', expand=True)

In [ ]:
df_c

Necessary and Sufficient Causes

In [ ]:
#Note: The openai-python library support for Azure OpenAI is in preview.
      #Note: This code sample requires OpenAI Python library version 0.28.1 or lower.
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://dap-open-ai-swc.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "1fc75eeca3934ae29b46d59cfda85968"
necessary_cause = "Causes"  # replace with actual necessary causes
sufficient_cause = "Causes"  # replace with actual sufficient causes

cause_dict = {"Necessary Cause": necessary_cause, "Sufficient Cause": sufficient_cause}

message_text = [{"role":"system",
"content":f"""You are an expert in Causal Reasoning (Actual Causality). Your aware of the INUS Causation.
Here are the patient details:
--- 
{patient_info}
---
Provide necessary and sufficient cause(s) for the patient's heart disease separately.
Answer with medical explantions in precise manner in the following format:
---
{cause_dict}
---
"""}]

completion = openai.ChatCompletion.create(
  engine="gpt-4-32k-deployment",
  messages = message_text,
  temperature=0.0,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

In [ ]:
print(completion.choices[0].message.content)

Responsibility

In [ ]:
#Note: The openai-python library support for Azure OpenAI is in preview.
      #Note: This code sample requires OpenAI Python library version 0.28.1 or lower.
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://dap-open-ai-swc.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "1fc75eeca3934ae29b46d59cfda85968"

message_text = [{"role":"system","content":f"""
In the context of this dialog, define “responsibility” as the relative degree to which an event causally
contributes to a particular outcome event, relative to
other events that caused that outcome. Given two
causal events that cause the same outcome event, one
causal event can be more, less, or equally responsible
relative to the other. In medical cases, the responsibility should be action-based.

Based on this definition of responsibility and the patient info:{patient_info}, answer precisely which factor is/are most responsible for his/her of heart disease."""}]

completion = openai.ChatCompletion.create(
  engine="gpt-4-32k-deployment",
  messages = message_text,
  temperature=0.0,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

In [ ]:
print(completion.choices[0].message.content)